In [26]:
import pandas as pd
import requests
import json
import os as os
import logging


In [44]:
pathforjsons = "C:/Users/prksh/Downloads/CLusteringTR/NSCLC_paridhy/json_response"
samples = "C:/Users/prksh/Downloads/CLusteringTR/NSCLC_paridhy/samples.txt"

In [45]:
os.chdir(pathforjsons)

In [5]:

body={
   "email" : "dinesh@4basecare.com",
   "password" :"dinesh@123"
}
token = requests.post("https://www.v2.api.ecrf.4basecare.co.in/user/login", json=body).json()['payLoad']['authToken']
patient = 'IN-423-VBVD'
url = f"https://www.v2.api.ecrf.4basecare.co.in/integration/getExternalApiResponseByPatientId/{patient}"
headers = {'Authorization': f'Bearer {token}'}
rs = requests.get(url, headers = headers).json()
rs

{'success': True,
 'message': 'successfully fetched',
 'payLoad': {'patientInfo': {'patientId': 'IN-423-VBVD',
   'patientUUId': 'f1ba8628-618c-4001-b130-60139390ee80',
   'bmi': None,
   'preMenopausecycleType': None,
   'preMenopausecycleDuration': None,
   'age': 52,
   'gender': 'FEMALE',
   'menstruation': None,
   'occupation': None,
   'ethnicity': None,
   'height': 164.0,
   'weight': 58.0,
   'diet': 'Vegetarian',
   'maritalStatus': None,
   'numberOfChildren': 2,
   'parity': None},
  'medicalInfo': {'approximateDate': None,
   'ageAtDiagnosis': 50,
   'symptomsBeforeDiagnosis': 'Coughing ',
   'hereditaryFactor': 'SIGNIFICANT',
   'pastHistoryFactor': None,
   'hereditaryFactorSignificant': None,
   'synchronousOrMetachronous': None,
   'hereditaries': [{'patientRelation': 'Father',
     'cancerSiteName': None,
     'ageOfDiagnosis': None}],
   'caseSummaryInfos': [{'reportedDate': '2022-02-16',
     'source': None,
     'hospitalNameCSD': None,
     'hospitalLocationNameC

In [29]:

api_url = "https://www.v2.api.ecrf.4basecare.co.in/integration/getExternalApiResponseByPatientId/"
pat_df = pd.read_csv(samples, sep='\t')
# List of patient IDs
patient_ids= pat_df['samples'].to_list()
open('data_export.log', 'a').close()
logging.basicConfig(filename='data_export.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
patient_ids[:5]

['IN-423-VL4B', 'IN-423-WGYAF', 'IN-423-WC3F', 'IN-423-VJJM', 'IN-423-WAIZ']

In [30]:
def fetch_and_export_data(patient_id, token):
    url = f"{api_url}{patient_id}"
    headers = {'Authorization': f'Bearer {token}'}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        # Parse the JSON response
        patient_data = response.json()
        
        # Export the patient data as a JSON file
        output_file = f"{patient_id}_data.json"
        with open(output_file, 'w') as json_file:
            json.dump(patient_data, json_file, indent=2)
        logging.info(f"Data for {patient_id} exported successfully.")
    else:
        logging.error(f"Error fetching data for {patient_id}. Status code: {response.status_code}")

In [31]:
# Loop through each patient ID and fetch/export data


for p in range(len(patient_ids)):
    fetch_and_export_data(patient_ids[p], token)
    print(f' {p} || {len(patient_ids)}', end="\r")

In [46]:
json_list=os.listdir(os.getcwd())
json_list=[i for i in json_list if i.endswith('.json')]
print(json_list[:5])

['34007P012_data.json', '34007P014_data.json', '34007P067_data.json', 'IN-423-TKKC_data.json', 'IN-423-TKZB_data.json']


In [33]:
def json_cd(info,path, output_path):
    df = pd.read_json(path)
    df.drop(['success', 'message'], axis=1, inplace=True)
    df=df.T
    df.to_json
    df[info].to_json(output_path, orient='records', indent=2)


In [47]:
info='patientInfo'
os.makedirs(info, exist_ok=True)
for i in range(len(json_list)):
    json_cd(info,json_list[i], f"{pathforjsons}/{info}/{json_list[i]}_{info}.json")
    print(f' {i} || {len(json_list)}', end="\r")
    
os.chdir(f"{pathforjsons}/{info}/")

In [48]:
cd_list = [file for file in os.listdir(os.getcwd()) if file.endswith('.json')]
cd_list[:5]

['34007P012_data.json_patientInfo.json',
 '34007P014_data.json_patientInfo.json',
 '34007P067_data.json_patientInfo.json',
 'IN-423-TKKC_data.json_patientInfo.json',
 'IN-423-TKZB_data.json_patientInfo.json']

# Test

In [27]:
from pandas.io.json import json_normalize
# Read the JSON file
with open('c:/Users/prksh/Downloads/CLusteringTR/Clustering_Pilot2/json_response/IN-423-UI3A_data.json') as f:
    json_data = json.load(f)

# Remove the outer square brackets
#json_data = json_data[0][0]

# Use json_normalize to flatten the nested structure
df = pd.json_normalize(json_data, sep='_')
df

,success,message,payLoad_patientInfo_patientId,payLoad_patientInfo_patientUUId,payLoad_patientInfo_bmi,payLoad_patientInfo_preMenopausecycleType,payLoad_patientInfo_preMenopausecycleDuration,payLoad_patientInfo_age,payLoad_patientInfo_gender,payLoad_patientInfo_menstruation,...,payLoad_medicalInfo_isMedicalAllergies,payLoad_medicalInfo_medicalAllergiesSpecify,payLoad_medicalInfo_immunosupression,payLoad_medicalInfo_imunosupressionSpecify,payLoad_medicalInfo_otherFormsOfTobaccoUse,payLoad_medicalInfo_otherFormsOfTobaccoUseSpecify,payLoad_medicalInfo_cancerDetailsComments,payLoad_medicalInfo_dateOfDiagnosis,payLoad_cancerDetails,payLoad_events
0,True,successfully fetched,IN-423-UI3A,5b46aee0-5b3f-4ff1-86ff-8527736c2f6b,26.4,None,None,64,MALE,None,...,True,Penicillin Allergy,Unknown,None,Unknown,None,None,None,"[{'cancerSiteIdcCode': 'C34', 'cancerSite': 'L...","[{'eventType': 'DIAGNOSTIC_TEST', 'eventDate':..."


In [49]:
merged_df=[]
for i in cd_list:
    with open(i) as f:
        json_data = json.load(f)
    # Remove the outer square brackets
    if not json_data or not json_data[0]:
        print("JSON structure is empty or does not contain any fields.")
    else:
     json_data = json_data[0]
     df = pd.json_normalize(json_data, sep='_')
     df['Sample'] = i.replace('_data.json_MD.json', '')
     merged_df.append(df)
    

In [28]:
if not json_data or not json_data[0]:
    print("JSON structure is empty or does not contain any fields.")

JSON structure is empty or does not contain any fields.


In [50]:
pd.concat(merged_df).to_csv(f"C:/Users/prksh/Downloads/CLusteringTR/nsclc_details_pd.csv")